In [1]:
import numpy as np 
import pandas as pd
import os
import sklearn as skl
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
from dataclasses import dataclass
import glob
import gc

Create read_data_file which gives wifi data, waypoint data, and siteID

In [2]:
# copy from https://github.com/location-competition/indoor-location-competition-20/blob/master/io_f.py

@dataclass
class ReadData:
    wifi: np.ndarray
    waypoint: np.ndarray
    siteID: str

def read_data_file(data_filename):
    wifi = []
    waypoint = []
    siteID = ''

    with open(data_filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for line_data in lines:
        
        if 'SiteID:' in line_data:
            siteID = line_data.split('\t')[1].replace('SiteID:', '')

        line_data = line_data.strip()
        if not line_data or line_data[0] == '#':
            continue

        line_data = line_data.split('\t')
        
        if line_data[1] == 'TYPE_WAYPOINT':
            waypoint.append([int(line_data[0]), float(line_data[2]), float(line_data[3])])
            continue
        
        if line_data[1] == 'TYPE_WIFI':
            sys_ts = line_data[0]
            ssid = line_data[2]
            bssid = line_data[3]
            rssi = line_data[4]
            lastseen_ts = line_data[6]
            wifi_data = [sys_ts, ssid, bssid, rssi, lastseen_ts]
            wifi.append(wifi_data)
            continue
    
    wifi = np.array(wifi)
    waypoint = np.array(waypoint)
    #print(wifi)
    
    return ReadData(wifi, waypoint, siteID)

sample_file = read_data_file("/Users/neel2sharma/indoor-location-navigation/train/5a0546857ecc773753327266/F2/5dccf516c04f060006e6e3c9.txt")

print('wifi shape:', sample_file.wifi.shape)
print('waypoint shape:', sample_file.waypoint.shape)

wifi shape: (5258, 5)
waypoint shape: (6, 3)


Filter the used buildings into used_buildings in order to remove unnecessary training files

In [24]:
#https://www.kaggle.com/devinanzelmo/wifi-features

# pull out all the buildings actually used in the test set, given current method we don't need the other ones
ssubm = pd.read_csv('/Users/neel2sharma/indoor-location-navigation/sample_submission.csv')

# only 24 of the total buildings are used in the test set, 
# this allows us to greatly reduce the intial size of the dataset

ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
used_buildings = sorted(ssubm_df[0].value_counts().index.tolist())

floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2, "F4":3, "F5":4, "F6":5, "F7":6,"F8":7, "F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5, "7F":6, "8F": 7, "9F":8}


Create 24 different train_x and 24 of each train_yx, train_yy
    
Note siteID and filename (which is path) somewhere - in a map?
    
Figure out how to predict floor

Create 24 models and train appropriately

In [17]:
MNBX1 = MultinomialNB()
MNBX2 = MultinomialNB()
MNBX3 = MultinomialNB()
MNBX4 = MultinomialNB()
MNBX5 = MultinomialNB()
MNBX6 = MultinomialNB()
MNBX7 = MultinomialNB()
MNBX8 = MultinomialNB()
MNBX9 = MultinomialNB()
MNBX10 = MultinomialNB()
MNBX11 = MultinomialNB()
MNBX12 = MultinomialNB()
MNBX13 = MultinomialNB()
MNBX14 = MultinomialNB()
MNBX15 = MultinomialNB()
MNBX16 = MultinomialNB()
MNBX17 = MultinomialNB()
MNBX18 = MultinomialNB()
MNBX19 = MultinomialNB()
MNBX20 = MultinomialNB()
MNBX21 = MultinomialNB()
MNBX22 = MultinomialNB()
MNBX23 = MultinomialNB()
MNBX24 = MultinomialNB()

In [23]:
MNBY1 = MultinomialNB()
MNBY2 = MultinomialNB()
MNBY3 = MultinomialNB()
MNBY4 = MultinomialNB()
MNBY5 = MultinomialNB()
MNBY6 = MultinomialNB()
MNBY7 = MultinomialNB()
MNBY8 = MultinomialNB()
MNBY9 = MultinomialNB()
MNBY10 = MultinomialNB()
MNBY11 = MultinomialNB()
MNBY12 = MultinomialNB()
MNBY13 = MultinomialNB()
MNBY14 = MultinomialNB()
MNBY15 = MultinomialNB()
MNBY16 = MultinomialNB()
MNBY17 = MultinomialNB()
MNBY18 = MultinomialNB()
MNBY19 = MultinomialNB()
MNBY20 = MultinomialNB()
MNBY21 = MultinomialNB()
MNBY22 = MultinomialNB()
MNBY23 = MultinomialNB()
MNBY24 = MultinomialNB()

In [19]:
MNBYY1 = MultinomialNB()
MNBYY2 = MultinomialNB()
MNBYY3 = MultinomialNB()
MNBYY4 = MultinomialNB()
MNBYY5 = MultinomialNB()
MNBYY6 = MultinomialNB()
MNBYY7 = MultinomialNB()
MNBYY8 = MultinomialNB()
MNBYY9 = MultinomialNB()
MNBYY10 = MultinomialNB()
MNBYY11 = MultinomialNB()
MNBYY12 = MultinomialNB()
MNBYY13 = MultinomialNB()
MNBYY14 = MultinomialNB()
MNBYY15 = MultinomialNB()
MNBYY16 = MultinomialNB()
MNBYY17 = MultinomialNB()
MNBYY18 = MultinomialNB()
MNBYY19 = MultinomialNB()
MNBYY20 = MultinomialNB()
MNBYY21 = MultinomialNB()
MNBYY22 = MultinomialNB()
MNBYY23 = MultinomialNB()
MNBYY24 = MultinomialNB()

In [ ]:
#have an array of MNBs - loop through each building and build train_x, train_yx, train_yy internally
#then do MNBXS[i].fit(train_x, train_yx), MNBYS[i].fit(train_x, train_yy)

#still need to figure out how to predict floor, how to link each model pair (MNBX1, MNBY1) to siteID (path is determined by the test file)
    #linking is important because it tells me which model to use for each test file

MNBXS = [MNBX1, MNBX2, MNBX3, MNBX4, MNBX5, MNBX6, MNBX7, MNBX8, MNBX9, MNBX10, MNBX11, MNBX12, MNBX13, MNBX14, MNBX15, MNBX16, MNBX17, MNBX18, MNBX19, MNBX20, MNBX21, MNBX22, MNBX23, MNBX24]
MNBYS = [MNBY1, MNBY2, MNBY3, MNBY4, MNBY5, MNBY6, MNBY7, MNBY8, MNBY9, MNBY10, MNBY11, MNBY12, MNBY13, MNBY14, MNBY15, MNBY16, MNBY17, MNBY18, MNBY19, MNBY20, MNBY21, MNBY22, MNBY23, MNBX24]

z = 0
for building in used_buildings:

    folders = sorted(glob.glob(os.path.join('/Users/neel2sharma/indoor-location-navigation/','train/'+building+'/*')))
    print(building)

    for folder in folders:
        floor = floor_map[folder.split('/')[-1]]
        files = glob.glob(os.path.join(folder, "*.txt"))

        train_x = []
        train_yx = []
        train_yy = []
        
        for file in files:
            #at this point, goes through each file - floor is stored in floor, siteID is stored in building, and path is file[80:104]
            #Now fill train_x, train_yx, and train_yy as before
            train_file = read_data_file(file)
            
            #obtain waypoint values
            waypoint_values_x = train_file.waypoint[:,1]
            waypoint_values_y = train_file.waypoint[:,2]
            waypoint_timestamps = train_file.waypoint[:,0]

            #Append x and y data for this file into train_yx and train_yy (lists rather than ndarrays for sklearn)
            i = 0
            while i < len(waypoint_values_x):
                train_yx.append(waypoint_values_x[i].astype(int))
                i += 1
                
            i = 0
            while i < len(waypoint_values_y):
                train_yy.append(waypoint_values_y[i].astype(int))
                i += 1
                
            #Select closest timestamps
            wifi_timestamps = [0] * len(waypoint_timestamps)
            i = 0
            while i < len(waypoint_timestamps):
                for x in train_file.wifi:
                    y = x[0].astype(float)
                    if abs(waypoint_timestamps[i] - y) < abs(waypoint_timestamps[i] - wifi_timestamps[i]):
                        wifi_timestamps[i] = y
                i += 1


            #create nparray of zeroes with len = 561 as toAppend. Instead of appending to toAppend, use i to update individual values
            #this way all arrays in train_x have same length
            i = 0
            while i < len(wifi_timestamps):
                toAppend = np.zeros(561)
                for x in train_file.wifi:
                    if wifi_timestamps[i].astype(float) == x[0].astype(float):
                        if x[3] == 'nan':
                            toAppend[i] = 0
                        else:
                            toAppend[i] = x[3].astype(int)
                toAppend[i] = floor #may be out of bounds, maybe do 562
                train_x.append(toAppend)
                i += 1

            list_train_x = np.array([np.array(xi) for xi in train_x])

            MNBXS[z].fit(train_x, train_yx)
            MNBYS[z].fit(train_x, train_yy)

    z += 1


Compile test data into 24 different test_x

Predict x and y

Compile into csv for submission

still need to figure out how to predict floor and timestamp

In [ ]:
#do test_x internally and store predictions in lists

sites = [] #check siteID
paths = [] #check file name
x_predict = [] #do I need 24 of these?
y_predict = []

for building in used_buildings: #alter this to fit test data 

    folders = sorted(glob.glob(os.path.join('/Users/neel2sharma/indoor-location-navigation/','train/'+building+'/*')))
    print(building)

    for folder in folders:
        

In [ ]:
test_x = []

for dirname, _, filenames in os.walk('/Users/neel2sharma/indoor-location-navigation/test'):
    for filename in filenames:
        if (filename == ".DS_Store"):
            continue
        train_file = read_data_file(os.path.join(dirname, filename))
        
       # if (train_file.siteID != '5c3c44b80379370013e0fd2b'):
        #    continue
        
        #problems here: wifi_timestamps doesn't exist - I don't know how many of the rssi rows to take
        #train_x looks like [[rssi11, rssi12, rssi13, rssi13, rssi14, rssi15, rssi16], [rssi21, rssi22, rssi23, rssi24, rssi25, rssi26], ...]
            #in the current filter, the longest such sublist is 561 points long
        i = 0
        while i < 6: #make a prediction for each timestamp
            toAppend = np.zeros(561) #I got 561 for the length of the longest rssi point
            
            if x[3] == 'nan':
                toAppend[i] = 0
            else:
                toAppend[i] = x[3].astype(int)
            test_x.append(toAppend)
            i += 1
    



#bnb and lr are giving lists of lists of the same value (ex [[97, 101], [97, 101], [97, 101],...])
#mnb is giving a 2D array where each diagonal line is repeats
    #honestly not sure what's going on - test data once prof responds, set up meeting if it fails
predicted_x = MNBX.predict(test_x)
predicted_y = MNBY.predict(test_x)

predicted_merge = []
i = 0
while (i < len(predicted_x)):
    toAppend = []
    toAppend.append(predicted_x[i])
    toAppend.append(predicted_y[i])
    predicted_merge.append(toAppend)
    i += 1
print(predicted_merge)